<a href="https://colab.research.google.com/github/AlexW0316/Sports-Analytics/blob/main/Copy_of_My_Elo_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
#Links to your google drive
from google.colab import drive
drive.mount('/content/drive')

#Imports data.csv to a pandas "dataframe"
df=pd.read_csv('/content/drive/MyDrive/Sports Analytics/Data/2019.csv')

Mounted at /content/drive


In [ ]:
#The first row
print(df.iloc[0])

df.iloc[0]['home_points']

id                                                       401110723
season                                                        2019
week                                                             1
season_type                                                regular
start_date                                2019-08-24T23:00:00.000Z
start_time_tbd                                                 NaN
neutral_site                                                  True
conference_game                                              False
attendance                                                 66543.0
venue_id                                                      4013
venue                                        Camping World Stadium
home_id                                                         57
home_team                                                  Florida
home_conference                                                SEC
home_points                                                   

24

In [ ]:
#Links to your google drive
from google.colab import drive
drive.mount('/content/drive')

#Imports data.csv to a pandas "dataframe"
df2=pd.read_csv('/content/drive/MyDrive/Sports Analytics/Data/FBS_data.csv')
#df2

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
name = df2.loc[:,"school"]
name = name.tolist()
#name

logistic to test: randomly picked some matches.
After week 4 < 14

In [ ]:
#Let's build a table to store ELO scores
elo_table=pd.DataFrame(np.zeros(shape=(len(name),16)))
elo_table.index=name
for i in range(16):
  elo_table[i]=1000 #initialize with 1000
#elo_table.loc[("Air Force",0)]=1200
print(elo_table)

                     0     1     2     3     4   ...    11    12    13    14    15
Air Force          1000  1000  1000  1000  1000  ...  1000  1000  1000  1000  1000
Akron              1000  1000  1000  1000  1000  ...  1000  1000  1000  1000  1000
Alabama            1000  1000  1000  1000  1000  ...  1000  1000  1000  1000  1000
Appalachian State  1000  1000  1000  1000  1000  ...  1000  1000  1000  1000  1000
Arizona            1000  1000  1000  1000  1000  ...  1000  1000  1000  1000  1000
...                 ...   ...   ...   ...   ...  ...   ...   ...   ...   ...   ...
Western Kentucky   1000  1000  1000  1000  1000  ...  1000  1000  1000  1000  1000
Western Michigan   1000  1000  1000  1000  1000  ...  1000  1000  1000  1000  1000
West Virginia      1000  1000  1000  1000  1000  ...  1000  1000  1000  1000  1000
Wisconsin          1000  1000  1000  1000  1000  ...  1000  1000  1000  1000  1000
Wyoming            1000  1000  1000  1000  1000  ...  1000  1000  1000  1000  1000

[13

In [ ]:
for index, row in df.iterrows():
  team1 = row["home_team"]
  team2 = row["away_team"]
  if(team1 not in name or team2 not in name):
    continue
  elo_1 = elo_table.loc[team1,row["week"]-1]
  elo_2 = elo_table.loc[team2,row["week"]-1]

  if row["neutral_site"]=="FALSE":
    elo_1 += 55

  elo_diff = elo_1 - elo_2
  prob1 = 1/(1 + 10 ** (-elo_diff / 400))
  prob2 = 1 - prob1

  if row["neutral_site"]=="FALSE":
    elo_1 -= 55

  k = 25
  if(row["away_points"] > row["home_points"]):
    for i in range(row["week"], 16):
      elo_table.loc[(team1, i)] = elo_1 + k*(0 - prob1) ## set k = 25
      elo_table.loc[(team2, i)] = elo_2 + k*(1 - prob2) ## set k = 25
  else:
    for i in range(row["week"], 16):
      elo_table.loc[(team1, i)] = elo_1 + k*(1 - prob1) ## set k = 25
      elo_table.loc[(team2, i)] = elo_2 + k*(0 - prob2)

print(elo_table)

                     0       1   ...           14           15
Air Force          1000  1000.0  ...  1075.931854  1075.931854
Akron              1000   987.5  ...   868.131048   868.131048
Alabama            1000  1012.5  ...  1067.723398  1067.723398
Appalachian State  1000  1000.0  ...  1090.313990  1102.129538
Arizona            1000   987.5  ...   945.430441   945.430441
...                 ...     ...  ...          ...          ...
Western Kentucky   1000  1000.0  ...  1048.866941  1048.866941
Western Michigan   1000  1000.0  ...  1007.628860  1007.628860
West Virginia      1000  1000.0  ...   974.434685   974.434685
Wisconsin          1000  1012.5  ...  1087.092573  1076.095278
Wyoming            1000  1012.5  ...  1006.983225  1006.983225

[130 rows x 16 columns]


In [ ]:
#test pseduo

import random, math
def test(table, typ):
  l = []
  for index, row in df.iterrows():
    team1 = row["home_team"]
    team2 = row["away_team"]
    if(team1 not in name or team2 not in name):
      continue

    if(row["week"] >= 4 and row["week"] < 14):
      rand = random.random()
      if(rand <= 0.2):
        l.append(row)
  if(typ == "my_elo"):
    predict = table.loc[:,15]
  else:
    predict = table["talent"]
  count = 0
  for row in l:
    team1 = row["home_team"]
    team2 = row["away_team"]
    point1 = row["home_points"]
    point2 = row["away_points"]


    if(typ == "my_elo"):
      elo_1 = predict[team1]
      elo_2 = predict[team2]
      elo_diff = elo_1 - elo_2
      prob1 = 1/(1 + 10 ** (-elo_diff / 400))
      if(point1 < point2):
          count += math.log(1-prob1)
      elif(point1 > point2):
          count += math.log(prob1)
    else:

      if(point1 < point2 and predict[team1] < predict[team2]):
        count +=1
      elif(point1 > point2 and predict[team1] > predict[team2]):
        count += 1

  if(typ == "my_elo"):
    print(-count/len(l))
  else:
    print(count/len(l))


test(elo_table, "my_elo")

0.5756480050358982


In [ ]:
#basic model
#composite ranking? using previous year
#elo_table initial ranking   use previous year training (train)
#real prob (train) adjust actual - predict(not 0 or 1)
#improve actual prob  win-lost?

df_talent=pd.read_csv('/content/drive/MyDrive/Sports Analytics/Data/talent_2018.csv')

#print(df_talent)
#print(name)

for i, row in df_talent.iterrows():
  if(row["school"] not in name):
    df_talent = df_talent.drop(index = i)

df_talent = df_talent.reset_index(drop = True)
print(df_talent)

     year            school  talent
0    2018        Ohio State  984.30
1    2018           Alabama  978.54
2    2018           Georgia  964.00
3    2018               USC  933.65
4    2018           Clemson  893.21
..    ...               ...     ...
125  2018  Coastal Carolina  284.24
126  2018  New Mexico State  249.32
127  2018              Army  230.75
128  2018         Air Force  186.27
129  2018           Liberty   93.99

[130 rows x 3 columns]


In [ ]:
index = df_talent["school"].tolist()
talent = df_talent["talent"].tolist()
df_basic = pd.DataFrame({'talent': talent},
                  index=index)
test(df_basic, "")

0.6444444444444445


In [ ]:
from torchvision.datasets import CIFAR10
from torchvision.transforms import transforms
from torch.utils.data import DataLoader

# Loading and normalizing the data.
# Define transformations for the training and test sets
transformations = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# CIFAR10 dataset consists of 50K training images. We define the batch size of 10 to load 5,000 batches of images.
batch_size = 10
number_of_labels = 10

# Create an instance for training.
# When we run this code for the first time, the CIFAR10 train dataset will be downloaded locally.
train_set =CIFAR10(root="./data",train=True,transform=transformations,download=True)

# Create a loader for the training set which will read the data within batch size and put into memory.
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=0)
print("The number of images in a training set is: ", len(train_loader)*batch_size)

# Create an instance for testing, note that train is set to False.
# When we run this code for the first time, the CIFAR10 test dataset will be downloaded locally.
test_set = CIFAR10(root="./data", train=False, transform=transformations, download=True)

# Create a loader for the test set which will read the data within batch size and put into memory.
# Note that each shuffle is set to false for the test loader.
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=0)
print("The number of images in a test set is: ", len(test_loader)*batch_size)

print("The number of batches per epoch is: ", len(train_loader))
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')